<a href="https://colab.research.google.com/github/kyochanpy/forest_bears/blob/main/make_df/08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tabula-py
!pip install geopandas
!pip install jismesh
!pip install folium

     |████████████████████████████████| 12.0 MB 4.5 MB/s 
     |████████████████████████████████| 1.0 MB 5.0 MB/s 
     |████████████████████████████████| 6.3 MB 40.2 MB/s 
     |████████████████████████████████| 15.4 MB 27.3 MB/s 


In [ ]:
import re

import pandas as pd
import numpy as np
import tabula
from tabula import read_pdf
import math
import matplotlib.pyplot as plt
import plotly.express as px

from datetime import datetime as dt

import geopandas as gpd
import jismesh.utils as ju
import folium
from shapely.geometry import Polygon
import branca.colormap as cm

tabula-pyを使って直接テーブルを取得しようとしたがセル結合・表記ブレからうまくいかなかったので、pdf→word→excelの順にテーブルをコピーして以下の関数を適用。

In [ ]:
def set_columns(input_df):
    input_df_ = input_df.copy()

    output_df = input_df_.shift()                
    output_df.iloc[0] = output_df.columns.values
    output_df.columns = ['No.', '区', '発生日時', '出没場所', '状況']

    return output_df

In [ ]:
def fix_df(input_df):
    input_df_ = input_df.copy()
    
    idx_list = []
    no_list = input_df_["No."].tolist()
    for i in no_list:
        if np.isnan(i):
            idx_list.append(no_list.index(i))

    for i in idx_list:
        i_ = input_df_.iloc[i, 2]
        if type(i_) != float:
            input_df_.iloc[i, 2]  = i_.strftime('%H:%M')

    input_df_ = input_df_.fillna("")

    for i in idx_list:
        input_df_.iloc[i-1, 2:4] = input_df_.iloc[i-1, 2:4] + input_df_.iloc[i, 2:4]
        
    output_df = input_df_.drop(idx_list)

    return output_df

In [ ]:
def make_df():
    output_df = pd.DataFrame()
    for i in range(5):
        excel_df = pd.read_excel("/content/drive/MyDrive/forest_bears/forest_bears.xlsx", sheet_name=i)
        tmp_df = set_columns(excel_df)
        df = fix_df(tmp_df)
        output_df = pd.concat([output_df, df])
    
    return output_df

ここでGoogle Map APIで住所を経緯度に変換。

In [ ]:
latlng_df = pd.read_csv('/content/drive/MyDrive/forest_bears/forest_bears_with_latlng.csv')

In [ ]:
latlng_df

,No.,区,発生日時,出没場所,状況,lat,lng
0,1.0,南区,平28年 4月14日(木),南区南沢市民の森 案内板2番と3番の間,足跡を確認,42.984113,141.301965
1,2.0,南区,平28年 4月20日(水),南区白川市民の森 案内板7番から8番へ30m地点 案内板5番から21番へ200m地点,足跡・爪痕を確認,42.983615,141.283236
2,3.0,南区,平28年 4月21日(木),南区定山渓920番地付近 豊羽鉱山への道道,足跡を確認,42.982008,141.041829
3,4.0,南区,平28年 4月28日(木),南区真駒内柏丘1丁目 真駒内川河川敷,ヒグマらしき動物を目撃,42.984386,141.348751
4,5.0,南区,平28年 5月14日(土),南区定山渓（定山渓自然の村南東約300mの林道）,ヒグマを目撃,42.931382,141.152064
...,...,...,...,...,...,...,...
558,91.0,手稲区,令和2年11月12日(木),手稲区手稲本町593番地151付近（手稲山麓西線上）,足跡・鳴き声を確認,43.077052,141.200096
559,92.0,西区,令和2年12月4日(金),西区小別沢20番地付近,足跡、被毛を確認,43.047359,141.277738
560,93.0,南区,令和2年12月23日(水)18:30,南区石山1045番地2付近,ヒグマを目撃,42.946191,141.328557
561,94.0,南区,令和3年3月25日(木),南区白川市民の森(案内板20番から17番方向へ約20mの付近),足跡を確認,42.983615,141.283236


In [ ]:
#和暦を西暦に直す関数
#発生日時の表記方法の特定に和暦を使ったため後に修正する必要がある
def wareki_to_seireki(input_list):
    output_list = []
    for i in input_list:
        if i == "h28":
            output_list.append(2016)
        elif i == "h29":
            output_list.append(2017)
        elif i == "h30":
            output_list.append(2018)
        elif i == "r1":
            output_list.append(2019)
        elif i == "r2":
            output_list.append(2020)
        else:
            output_list.append(2021)
        
    return output_list

In [ ]:
def get_date_time_features(input_df):
    input_df_ = input_df.copy()

    no_values = input_df_["No."].values
    len_list_ = []
    for i in range(len(no_values)+1):
        if no_values[i+1] - no_values[i] != 1:
            len_list_.append(i)
            if len(len_list_) == 4:
                break
    len_list_.append(len(no_values))
    
    len_list = []
    for i in range(0,len(len_list_)):
        i_ = len_list_[i] - len_list_[i-1]
        len_list.append(i_)
    len_list[0] = len_list_[0]

    fiscal_year_list = []
    for i, j in zip(
        len_list,
        ["h28", "h29", "h30", "r1", "r2"]
    ):
        fiscal_year_list = fiscal_year_list + [j] * i
    
    year_list = []
    month_list = []
    day_list = []
    day_week_list = []
    time_list = []
    for i in input_df_["発生日時"].values:
        i = i.replace(" ", "")
        if len(i) > 14:
            i_list = re.findall(r"\d+", i)
            if i_list[0] == "28":
                year_list.append(f"h{i_list[0]}")
            else:
                year_list.append(f"r{i_list[0]}")
            month_list.append(i_list[1])
            day_list.append(i_list[2])
            day_week_list.append(i[-7])
            time_list.append(i[-5:])
        elif len(i) <= 10:
            i_list = re.findall(r"\d+", i)
            year_list.append("不明")
            month_list.append(i_list[0])
            day_list.append(i_list[1])
            day_week_list.append(i[-2])
            time_list.append("不明")
        elif (len(i) > 10) and (i[-3] == ":"):
            i_list = re.findall(r"\d+", i)
            year_list.append("不明")
            month_list.append(i_list[0])
            day_list.append(i_list[1])
            day_week_list.append(i[-7])
            time_list.append(i[-5:])
        else:
            i_list = re.findall(r"\d+", i)
            if i_list[0] == "28":
                year_list.append(f"h{i_list[0]}")
            else:
                year_list.append(f"r{i_list[0]}")
            month_list.append(i_list[1])
            day_list.append(i_list[2])
            day_week_list.append(i[-2])
            time_list.append("不明")

    output_df = pd.DataFrame()
    output_df["年度"] = fiscal_year_list
    output_df["年"] = year_list
    output_df["月"] = month_list
    output_df["日"] = day_list
    output_df["曜日"] = day_week_list
    output_df["時間"] = time_list
   
    fixed_year_list = []
    for f, y, m in zip(
        output_df["年度"].values,
        output_df["年"].values,
        output_df["月"].values
    ):
        if y != "不明":
            fixed_year_list.append(y)
        else:
            if 4 <= int(m) <= 12:
                fixed_year_list.append(f)
            else:
                if y == "r2":
                    fixed_year_list.append("r3")
                else:
                    fixed_year_list.append(fixed_year_list[fixed_year_list.index(y)+1])
    output_df["年"] = fixed_year_list  

    for c in ["年度", "年"]:
        output_df[c] = wareki_to_seireki(output_df[c])

    return output_df

In [ ]:
df = get_date_time_features(latlng_df)

In [ ]:
df

,年度,年,月,日,曜日,時間
0,2016,2016,4,14,木,不明
1,2016,2016,4,20,水,不明
2,2016,2016,4,21,木,不明
3,2016,2016,4,28,木,不明
4,2016,2016,5,14,土,不明
...,...,...,...,...,...,...
558,2020,2020,11,12,木,不明
559,2020,2020,12,4,金,不明
560,2020,2020,12,23,水,18:30
561,2020,2021,3,25,木,不明


In [ ]:
def merge_weather_df(path_list):
    output_df = pd.DataFrame()

    for path in path_list:
        df = pd.read_csv(path, encoding="SHIFT-JIS")
        df_ = df.shift()
        df_.iloc[0] = df_.columns.values
        df_.columns = ["年", "月", "日", "平均気温", "日照時間", "日照ダミー", "合計全天日射量", "合計降雪量", "降雪ダミー", 
                       "平均風速", "最多風向", "平均蒸気圧", "平均現地気圧", "平均湿度", "平均雲量",
                       "天候（昼）", "天候（夜）", "最深積雪", "積雪ダミー", "合計降水量", "降水ダミー"]
        output_df = pd.concat([output_df, df_])
    
    return output_df


def fix_weather_data(input_df):
    output_df = input_df.copy()

    # concat時に崩れたダミー特徴量を修正
    for i in ["日照ダミー", "降雪ダミー", "積雪ダミー", "降水ダミー"]:
        output_df[i] = [i_[0] for i_ in output_df[i].values.astype(str)]

    # columnsからshiftさせるときに崩れた部分を修正
    for i in ["月", "日", "合計降雪量"]:
        list_ = []
        for i_ in output_df[i]:
            if type(i_) == str:
                list_.append(int(i_[0]))
            else:
                list_.append(i_)
        output_df[i] = list_

    # object, floatをintに変換
    for c in ["年", "月", "日"]:
        output_df[c] = output_df[c].astype(int)
    
    return output_df

In [ ]:
def check_weather_str(input_str):
    if "時々" in input_str:
        tmp_str = input_str.split("時々")[0]
    else:
        tmp_str = input_str

    if "一時" in tmp_str:
        tmp_str = tmp_str.split("一時")[0]
    else:
        tmp_str = tmp_str
    
    if "、" in tmp_str:
        tmp_str = tmp_str.split("、")[0]
    else:
        tmp_str = tmp_str

    if "\u3000" in tmp_str:
        output_str = tmp_str.replace("\u3000", "")
    else:
        output_str = tmp_str
    
    return output_str


def simple_weather(input_list):
    output_list = []
    for i in input_list:
        if "晴" in i:
            i_ = "晴"
        elif "雨" in i:
            i_ = "雨"
        elif "曇" in i:
            i_ = "曇"
        else:
            i_ = "雪"
        output_list.append(i_)
    
    return output_list


def make_weather_features(input_df):
    input_df_ = input_df.copy()

    output_df = input_df_[["年", "月", "日", "平均気温"]]
    for c in ["天候（昼）", "天候（夜）"]:
        w_1 = []
        w_2 = []
        for i in input_df_[c]:
            if "後" in i:
                i_1 = i.split("後")[0]
                i_2 = i.split("後")[1]
                w_1.append(check_weather_str(i_1))
                w_2.append(check_weather_str(i_2))
            else:
                w_1.append(check_weather_str(i))
                w_2.append(check_weather_str(i))
        w_1_ = []
        w_2_ = []
        for i_1_, i_2_ in zip(
            w_1,
            w_2
        ):
            if i_2_ == "":
                i_2_ = i_1_
            else:
                i_2_ = i_2_
            w_1_.append(i_1_)
            w_2_.append(i_2_)  
        output_df[f"{c}_1"] = simple_weather(w_1_)
        output_df[f"{c}_2"] = simple_weather(w_2_)

    return output_df

In [94]:
latlng_df

,No.,区,発生日時,出没場所,状況,lat,lng
0,1.0,南区,平28年 4月14日(木),南区南沢市民の森 案内板2番と3番の間,足跡を確認,42.984113,141.301965
1,2.0,南区,平28年 4月20日(水),南区白川市民の森 案内板7番から8番へ30m地点 案内板5番から21番へ200m地点,足跡・爪痕を確認,42.983615,141.283236
2,3.0,南区,平28年 4月21日(木),南区定山渓920番地付近 豊羽鉱山への道道,足跡を確認,42.982008,141.041829
3,4.0,南区,平28年 4月28日(木),南区真駒内柏丘1丁目 真駒内川河川敷,ヒグマらしき動物を目撃,42.984386,141.348751
4,5.0,南区,平28年 5月14日(土),南区定山渓（定山渓自然の村南東約300mの林道）,ヒグマを目撃,42.931382,141.152064
...,...,...,...,...,...,...,...
558,91.0,手稲区,令和2年11月12日(木),手稲区手稲本町593番地151付近（手稲山麓西線上）,足跡・鳴き声を確認,43.077052,141.200096
559,92.0,西区,令和2年12月4日(金),西区小別沢20番地付近,足跡、被毛を確認,43.047359,141.277738
560,93.0,南区,令和2年12月23日(水)18:30,南区石山1045番地2付近,ヒグマを目撃,42.946191,141.328557
561,94.0,南区,令和3年3月25日(木),南区白川市民の森(案内板20番から17番方向へ約20mの付近),足跡を確認,42.983615,141.283236


In [166]:
def get_geometry_features(input_df, path):
    input_df_ = input_df.copy()
    geo_df = gpd.read_file(path)

    geo_df["KEY_CODE"] = geo_df["KEY_CODE"].astype(int)

    output_df = pd.merge(input_df_, geo_df.iloc[:, [0,-1]], on="KEY_CODE", how="left")

    return output_df

In [175]:
def get_mesh5_features(input_df):
    input_df_ = input_df.copy()

    mesh5_list = []
    for lat, lng in zip(
        input_df_["lat"].to_numpy(),
        input_df_["lng"].to_numpy()
    ):
        mesh_code = ju.to_meshcode(lat, lng, 5)
        mesh5_list.append(mesh_code)
    
    output_df = input_df.copy()
    output_df["KEY_CODE"] = mesh5_list

    return output_df


def get_population_mesh5_features(path):
    df = pd.read_table(path, sep='\s+|,', engine='python')

    output_df = df.loc[:, ("KEY_CODE", "T000876001")].drop(0)
    output_df["KEY_CODE"] = output_df["KEY_CODE"].astype(int)
    output_df.rename(columns={"T000876001":"人口"}, inplace=True)
    output_df["人口"] = output_df["人口"].astype(int)
    return output_df


def make_mesh5_df(input_df, population_path, geo_path, weather_path_list):
    input_df_ = input_df.copy()

    #Geocodingがうまくいかなかったと思われる座標（No.97）を手動で修正。
    input_df_["lat"] = input_df_["lat"].replace(42.6033477, 43.0033312)
    input_df_["lng"] = input_df_["lng"].replace(140.6160604, 141.3402989)

    #地域メッシュコードを追加。
    input_df_mesh = get_mesh5_features(input_df_)

    #日時の特徴量。
    date_time_features = get_date_time_features(input_df_mesh)
    
    #天候の特徴量。
    
    weather_features_ = fix_weather_data(merge_weather_df(weather_path_list))
    weather_features = make_weather_features(weather_features_)

    #人口の特徴量（"KEY_CODE"有り）。
    population_features = get_population_mesh5_features(population_path)

    #特徴量をくっつける。
    output_df = pd.concat([input_df_mesh, date_time_features], axis=1)
    output_df = pd.merge(output_df, population_features, on="KEY_CODE", how="left")
    output_df = pd.merge(output_df, weather_features, on=["年", "月", "日"], how="left")
    output_df["人口"] = output_df[["人口"]].fillna(0).astype(int)
    
    #geometry特徴量をmerge。
    output_df = get_geometry_features(output_df, geo_path)

    return output_df

In [182]:
def get_mesh4_features(input_df):
    input_df_ = input_df.copy()

    mesh4_list = []
    for lat, lng in zip(
        input_df_["lat"].to_numpy(),
        input_df_["lng"].to_numpy()
    ):
        mesh_code = ju.to_meshcode(lat, lng, 4)
        mesh4_list.append(mesh_code)
    
    output_df = input_df.copy()
    output_df["KEY_CODE"] = mesh4_list

    return output_df


def get_population_mesh4_features(path):
    df = pd.read_table(path, sep='\s+|,', engine='python')

    output_df = df.loc[:, ("KEY_CODE", "T000847001")].drop(0)
    output_df["KEY_CODE"] = output_df["KEY_CODE"].astype(int)
    output_df.rename(columns={"T000847001":"人口"}, inplace=True)
    output_df["人口"] = output_df["人口"].astype(int)
    return output_df


def make_mesh4_df(input_df, population_path, geo_path, weather_path_list):
    input_df_ = input_df.copy()

    #Geocodingがうまくいかなかったと思われる座標（No.97）を手動で修正。
    input_df_["lat"] = input_df_["lat"].replace(42.6033477, 43.0033312)
    input_df_["lng"] = input_df_["lng"].replace(140.6160604, 141.3402989)

    #地域メッシュコードを追加。
    input_df_mesh = get_mesh4_features(input_df_)

    #日時の特徴量。
    date_time_features = get_date_time_features(input_df_)
    
    #天候の特徴量。
    
    weather_features_ = fix_weather_data(merge_weather_df(weather_path_list))
    weather_features = make_weather_features(weather_features_)

    #人口の特徴量（"KEY_CODE"有り）。
    population_features = get_population_mesh4_features(population_path)

    #特徴量をくっつける。
    output_df = pd.concat([input_df_mesh, date_time_features], axis=1)
    output_df = pd.merge(output_df, population_features, on="KEY_CODE", how="left")
    for c in ["年", "月", "日"]:
        output_df[c] = output_df[c].astype(int)
    output_df = pd.merge(output_df, weather_features, on=["年", "月", "日"])
    output_df["人口"] = output_df[["人口"]].fillna(0).astype(int)
    
    #geometry特徴量をmerge。
    output_df = get_geometry_features(output_df, geo_path)

    return output_df

In [161]:
def GDF(input_df):
    input_df_ = input_df.copy()
    output_df = gpd.GeoDataFrame(input_df_)
    return output_df

In [183]:
df = make_mesh4_df(latlng_df, 
                       "/content/drive/MyDrive/forest_bears/2015_6441.txt", 
                       "/content/drive/MyDrive/forest_bears/MESH06441_4",
                       ["/content/drive/MyDrive/forest_bears/weather_2016-2017.csv",
                        "/content/drive/MyDrive/forest_bears/weather_2018-2019.csv",
                        "/content/drive/MyDrive/forest_bears/weather_2020-2021.csv"])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [184]:
df

,No.,区,発生日時,出没場所,状況,lat,lng,KEY_CODE,年度,年,月,日,曜日,時間,人口,平均気温,天候（昼）_1,天候（昼）_2,天候（夜）_1,天候（夜）_2,geometry
0,1.0,南区,平28年 4月14日(木),南区南沢市民の森 案内板2番と3番の間,足跡を確認,42.984113,141.301965,644132841,2016,2016,4,14,木,不明,67,11,曇,曇,雨,雨,"POLYGON ((141.30625 42.98333, 141.30000 42.983..."
1,2.0,南区,平28年 4月20日(水),南区白川市民の森 案内板7番から8番へ30m地点 案内板5番から21番へ200m地点,足跡・爪痕を確認,42.983615,141.283236,644132822,2016,2016,4,20,水,不明,0,9.7,晴,曇,曇,晴,"POLYGON ((141.28750 42.98333, 141.28125 42.983..."
2,3.0,南区,平28年 4月21日(木),南区定山渓920番地付近 豊羽鉱山への道道,足跡を確認,42.982008,141.041829,644130733,2016,2016,4,21,木,不明,0,13.8,晴,曇,曇,曇,"POLYGON ((141.04375 42.97917, 141.03750 42.979..."
3,4.0,南区,平28年 4月28日(木),南区真駒内柏丘1丁目 真駒内川河川敷,ヒグマらしき動物を目撃,42.984386,141.348751,644132872,2016,2016,4,28,木,不明,1417,7.6,曇,曇,雨,雨,"POLYGON ((141.35000 42.98333, 141.34375 42.983..."
4,5.0,南区,平28年 5月14日(土),南区定山渓（定山渓自然の村南東約300mの林道）,ヒグマを目撃,42.931382,141.152064,644131123,2016,2016,5,14,土,不明,0,14.5,晴,晴,曇,曇,"POLYGON ((141.15625 42.92917, 141.15000 42.929..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,91.0,手稲区,令和2年11月12日(木),手稲区手稲本町593番地151付近（手稲山麓西線上）,足跡・鳴き声を確認,43.077052,141.200096,644141961,2020,2020,11,12,木,不明,0,4.3,晴,晴,晴,曇,"POLYGON ((141.20625 43.07500, 141.20000 43.075..."
559,92.0,西区,令和2年12月4日(金),西区小別沢20番地付近,足跡、被毛を確認,43.047359,141.277738,644142523,2020,2020,12,4,金,不明,6,0.8,晴,曇,曇,曇,"POLYGON ((141.28125 43.04583, 141.27500 43.045..."
560,93.0,南区,令和2年12月23日(水)18:30,南区石山1045番地2付近,ヒグマを目撃,42.946191,141.328557,644132363,2020,2020,12,23,水,18:30,41,1.3,晴,晴,曇,曇,"POLYGON ((141.33125 42.94583, 141.32500 42.945..."
561,94.0,南区,令和3年3月25日(木),南区白川市民の森(案内板20番から17番方向へ約20mの付近),足跡を確認,42.983615,141.283236,644132822,2020,2021,3,25,木,不明,0,9.5,曇,曇,雨,雨,"POLYGON ((141.28750 42.98333, 141.28125 42.983..."


In [185]:
df.to_csv('all_df.csv', index=False)